# Geo Code 

In [ ]:
import pandas as pd
import requests
# from geopy.distance import geodesic
# import re # regex
# import urllib.parse
import time

## Importing data

In [ ]:
%%time
data = pd.read_excel('data/PMTS_2019.xlsx', sheet_name=0)
data.shape

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
# Tranformar o campo Latitude em float
data['LATITUDE'] = data['LATITUDE'].apply(lambda x: str(x).replace(',', '.')).apply(lambda x: float(x))

In [ ]:
data.dtypes

## Geo location function

In [ ]:
# https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
from key import GoogleKey

key = GoogleKey().getKey()

In [ ]:
def getAddress(lat, lng):
    googleUrl = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='
    response = requests.get(googleUrl + str(lat) + ',' + str(lng) + '&key=' + key)    
    resp_json_payload = response.json()
    try:
        address     = resp_json_payload['results'][0]['formatted_address']
        cep         = re.findall(r"\d+\-\d+", address)
        latitude    = resp_json_payload['results'][0]['geometry']['location']['lat']
        longitude   = resp_json_payload['results'][0]['geometry']['location']['lng']

        if cep == []:
            cep = ['00000-000']
        # end if

        return (address, cep[0], latitude, longitude)
        
    except ValueError as ve:
        print(ve)
    except :
#         print('error', resp_json_payload)
        return ('not found', '00000-000', 0, 0)
#     finally:
        
#         print('error')
    # end try


In [ ]:
# -23.6088081,-46.6965317
getAddress(-23.6088081,-46.6965317)

In [ ]:
# -9.985957599999999, -67.8483179

getAddress(-9968417.0, -9968417.0)

## Consulta na base

In [ ]:
def geoBase(df):
    geo_list = []
    dfAux = df.copy()
    for index, row in dfAux.iterrows() :
        lat = str(row['LATITUDE'])
        lng = str(row['LONGITUDE'])
        data = getAddress(lat, lng)
        
        print(row['SITE ID'], data)
        geo_dictionary = {}
        
        geo_dictionary['SITE ID'] = row['SITE ID']
        geo_dictionary['PMTS ID'] = row['PMTS ID']
        geo_dictionary['address'] = str(data[0])
        geo_dictionary['cep'] = str(data[1])
        geo_dictionary['latitude'] = str(data[2])
        geo_dictionary['longitude'] = str(data[2])

        geo_list.append(geo_dictionary)
    

    geo_DataFrame = pd.DataFrame(geo_list)    

#     df['cep'] = geo_DataFrame['cep']
#     df['latitude'] = geo_DataFrame['latitude']
#     df['longitude'] = geo_DataFrame['longitude']
    
    return geo_DataFrame

In [ ]:
%%time
dfGeo = geoBase(data)

In [ ]:
dfGeo.head()

In [ ]:
dfGeo.to_csv('data/googleGeo.csv', sep=';')

In [ ]:
data['Google_CEP'] = dfGeo['cep']
data['Google_Endereco'] = dfGeo['address']

In [ ]:
data.head()

In [ ]:
print('Sites com o CEP não encontrado {0}'.format(data[data['Google_CEP'] == '00000-000'].shape[0]))
print('Sites com endereços não encontrados {0}'.format(data[data['Google_Endereco'] == 'not found'].shape[0]))

In [ ]:
data.head()

In [ ]:
data.to_excel('data/PMTS_2019_Google.xlsx', sheet_name='SitesGoogle')